In [175]:
import re
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time
import json
import pandas as pd

In [121]:
first_search = "https://www.immoweb.be/en/search/apartment/for-sale/st-josse-ten-noode/1210?countries=BE&priceType=SALE_PRICE&page=4&orderBy=relevance"

service = Service(executable_path="chromedriver_win.exe")

driver = webdriver.Chrome(service=service)

driver.implicitly_wait(3)

driver.get(first_search)

driver.maximize_window()

top_div = driver.find_element(By.ID, 'usercentrics-root')
shadow_root = top_div.shadow_root
confirm_button = shadow_root.find_element(By.CSS_SELECTOR, "button[data-testid='uc-accept-all-button']")

confirm_button.click()



In [135]:
def scrap_commune_urls(starting_point):
    url_list = []
    service = Service(executable_path="chromedriver_win.exe")
    driver = webdriver.Chrome(service=service)
    driver.implicitly_wait(3)
    driver.get(starting_point)
    driver.maximize_window()

    top_div = driver.find_element(By.ID, 'usercentrics-root')
    shadow_root = top_div.shadow_root
    confirm_button = shadow_root.find_element(By.CSS_SELECTOR, "button[data-testid='uc-accept-all-button']")

    confirm_button.click()

    ulist = driver.find_element(By.CLASS_NAME, "pagination")

    litems = ulist.find_elements(By.TAG_NAME, "li")

    pages_number = int(litems[-2].find_elements(By.TAG_NAME, "span")[1].text)

    for i in range(pages_number):        
        soup = bs(driver.page_source, 'html.parser')
        main_list = soup.find('ul', id='main-content')
        links = main_list.find_all('a')
        for link in links:
            url = link['href']
            if "immoweb.be" in url:
                url_list.append(url)        
        
        if i < pages_number - 1:            
            litems[-1].click()
            time.sleep(2)

    return url_list

    

In [177]:
def scrap_commune(driver, zip_code, com_name, property_type):
    url_list = []
    ulist = driver.find_element(By.CLASS_NAME, "pagination")
    litems = ulist.find_elements(By.TAG_NAME, "li")
    pages_number = int(litems[-2].find_elements(By.TAG_NAME, "span")[1].text)

    for i in range(pages_number):        
        soup = bs(driver.page_source, 'html.parser')
        main_list = soup.find('ul', id='main-content')
        links = main_list.find_all('a')
        for link in links:
            url = link['href']
            if "immoweb.be" in url:
                url_list.append(url)        
        
        if i < pages_number - 1:                        
            litems[-1].click()
            time.sleep(1.5)

    url_book = {'zip_code': zip_code, 'com_name': com_name, 'urls': url_list}

    url_book_json = json.dumps(url_book, indent=4)
    with open(f'url_store/{com_name}_{property_type}.json', 'w') as save_file:
        save_file.write(url_book_json)


    return url_list

In [139]:
def init_driver(start_url):
    service = Service(executable_path="chromedriver_win.exe")

    driver = webdriver.Chrome(service=service)

    driver.implicitly_wait(5)

    driver.get(start_url)

    driver.maximize_window()

    top_div = driver.find_element(By.ID, 'usercentrics-root')
    shadow_root = top_div.shadow_root
    confirm_button = shadow_root.find_element(By.CSS_SELECTOR, "button[data-testid='uc-accept-all-button']")

    confirm_button.click()



In [151]:
def change_commune(zip_to_go, driver):
    form = driver.find_element(By.CLASS_NAME, "multiselect__form")

    inputter = form.find_element(By.TAG_NAME, "input")

    remove_button = form.find_element(By.TAG_NAME, "button")

    remove_button.click()

    inputter.send_keys(zip_to_go)
    time.sleep(0.3)
    inputter.send_keys(Keys.ARROW_DOWN)
    time.sleep(0.3)
    inputter.send_keys(Keys.ENTER)


In [173]:
def bot_session(starting_point):
    service = Service(executable_path="chromedriver_win.exe")

    driver = webdriver.Chrome(service=service)

    driver.implicitly_wait(5)

    driver.get(starting_point)

    driver.maximize_window()

    top_div = driver.find_element(By.ID, 'usercentrics-root')
    shadow_root = top_div.shadow_root
    confirm_button = shadow_root.find_element(By.CSS_SELECTOR, "button[data-testid='uc-accept-all-button']")

    confirm_button.click()
    
    scrap_commune(driver, "1210", "St-Joost", "appartements")

    change_commune("1000", driver)
    time.sleep(2)

    scrap_commune(driver, "1000", "Bussels", "appartements")

    time.sleep(5)

In [174]:
bot_session("https://www.immoweb.be/en/search/apartment/for-sale/st-josse-ten-noode/1210?countries=BE&page=1&orderBy=relevance")

In [ ]:
#driver, zip_code, com_name, property_type

In [11]:
def immo_page_scrapper(source_html, locality=None, zip_code=None, property_type=0, property_subtype=None):
    real_estate = {'locality': locality, 'zip_code': zip_code, 'type_of_property': property_type, 'subtype_of_property': property_subtype, 'Price': None, 'building_condition': None, 'facade_number': None, 'living_area': None, 
               'equipped_kitchen': 0, 'bedroom_nr': None, 'swimming_pool': 0, 'furnished': 0, 'open_fire': 0, 'Terrace': 0, 'garden': 0, 'plot_surface': None}

    soup = bs(source_html, 'html.parser')

    second_headers = soup.find_all('h2')

    for second_h in second_headers:        
        if second_h.string == "Description":
            print('this is a description')
            paragraphs = second_h.parent.find_all('p')
            for par in paragraphs:
                if "open fire" in par.text:
                    real_estate['open_fire'] = 1

        elif second_h.string == "General":            
            rows = second_h.parent.parent.find_all('tr')
            for row in rows:                
                for cell in row.children:
                    if "Building condition" in cell.text:                        
                        real_estate['building_condition'] = (cell.find_next_sibling('td').text.strip())
                    elif "Number of frontages" in cell.text:                        
                        real_estate['facade_number'] = int(cell.find_next_sibling('td').text.strip())

        elif second_h.string == "Interior":
            rows = second_h.parent.parent.find_all('tr')
            for row in rows:
                for cell in row.children:
                    if "Living area" in cell.text:
                        liv_area = re.findall("(\d+)", cell.find_next_sibling('td').text)[0]
                        real_estate['living_area'] = int(liv_area)
                    elif "Kitchen type" in cell.text and "equipped" in cell.find_next_sibling('td').text:
                        real_estate['equipped_kitchen'] = 1
                    elif "Bedrooms" in cell.text:
                        real_estate['bedroom_nr'] = int(cell.find_next_sibling('td').text.strip())
                    elif "Furnished" in cell.text and "Yes" in cell.find_next_sibling('td').text:
                        real_estate['Furnished'] = 1

        elif second_h.string == "Exterior":
            rows = second_h.parent.parent.find_all('tr')
            for row in rows:
                for cell in row.children:
                    if "Surface of the plot" in cell.text:
                        plot_area = re.findall("(\d+)", cell.find_next_sibling('td').text)[0]
                        real_estate['plot_surface'] = plot_area
                    elif "Garden surface" in cell.text:
                        garden_area = re.findall("(\d+)", cell.find_next_sibling('td').text)[0]
                        real_estate['garden'] = garden_area
                    elif "Terrace surface" in cell.text:
                        terrace_area = re.findall("(\d+)", cell.find_next_sibling('td').text)[0]
                        real_estate['Terrace'] = terrace_area

        elif second_h.string == "Facilities":
            rows = second_h.parent.parent.find_all('tr')
            for row in rows:
                for cell in row.children:
                    if "Swimming pool" in cell.text and "Yes" in cell.find_next_sibling('td').text:
                        real_estate['swimming_pool'] = 1
                    

        elif second_h.string == "Financial":
            wrap_div = second_h.parent.parent
            rows = wrap_div.find_all('tr')
            for row in rows:
                for cell in row.children:
                    if "Price" in cell.text:
                        expr = re.compile("\d+\s€")
                        price_strings = re.findall(expr, cell.find_next_sibling('td').text)[0]
                        price_int = int(price_strings[:-2])
                        real_estate['Price'] = price_int

        

    return real_estate

In [12]:
test_to_scrap = "https://www.immoweb.be/en/classified/apartment-block/for-sale/bruxelles/1000/20010594"

service = Service(executable_path="chromedriver_win.exe")

driver = webdriver.Chrome(service=service)

driver.implicitly_wait(5)

driver.get(test_to_scrap)

driver.maximize_window()

top_div = driver.find_element(By.ID, 'usercentrics-root')
shadow_root = top_div.shadow_root
confirm_button = shadow_root.find_element(By.CSS_SELECTOR, "button[data-testid='uc-accept-all-button']")

confirm_button.click()

pagina = driver.page_source
imo = immo_page_scrapper(pagina)

print(imo)

this is a description
{'locality': None, 'zip_code': None, 'type_of_property': 0, 'subtype_of_property': None, 'Price': 800000, 'building_condition': 'Good', 'facade_number': 2, 'living_area': 250, 'equipped_kitchen': 1, 'bedroom_nr': 5, 'swimming_pool': 0, 'furnished': 0, 'open_fire': 0, 'Terrace': 0, 'garden': 0, 'plot_surface': '65'}


In [ ]:
"""
Locality +
Type of property (House/apartment) - binary is House +
Subtype of property (Bungalow, Chalet, Mansion, ...) +
Price +
Type of sale (Exclusion of life sales)
Number of rooms +
Living Area + 
Fully equipped kitchen (Yes/No) + 
Furnished (Yes/No) +
Open fire (Yes/No) +
Terrace (Yes/No) 
If yes: Area +
Garden (Yes/No) +
If yes: Area +
Surface of the land (+)
Surface area of the plot of land +
Number of facades +
Swimming pool (Yes/No) +
State of the building (New, to be renovated, ...) +
"""